In [1]:
from scipy import io
import pandas as pd
import ezodf

def read_ods(filename, sheet_no=0, header=0):
    tab=ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                        for col in tab.columns()})

In [2]:
df = read_ods(filename='/home/secl00/Documents/db1.ods')

In [3]:
df

,PROJECT,GUID,ELEMENTID,JOBNAME,FROM_NODE,TO_NODE,DELTA_X,DELTA_Y,DELTA_Z,DIAMETER,WALL_THICK,CORR_ALLOW,PRESSURE1,TEMP_MAX,LENGTH,MAXF,MAXT,SEG
0,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,1.0,810-P1004-S3,110.0,120.0,0.000000,102.000000,0.000000,219.099991,8.179,1.5,0.0,28.1,102.000000,0.000000,0.000000,301.0
1,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,2.0,810-P1004-S3,120.0,130.0,0.000000,305.000000,0.000000,219.099991,8.179,1.5,0.0,28.1,305.000000,5.926230,3.650530,301.0
2,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,3.0,810-P1004-S3,130.0,140.0,-454.999969,0.000000,0.000000,219.099991,8.179,1.5,0.0,28.1,454.999969,1.840538,3.131613,301.0
3,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,4.0,810-P1004-S3,140.0,150.0,-1444.099976,0.000000,0.000000,219.099991,8.179,1.5,0.0,28.1,1444.099976,3.131613,3.608645,301.0
4,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,5.0,810-P1004-S3,150.0,160.0,0.000000,0.000000,500.000000,219.099991,8.179,1.5,0.0,28.1,500.000000,3.350161,5.180240,301.0
5,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,6.0,810-P1004-S3,160.0,170.0,0.000000,0.000000,700.000000,219.099991,8.179,1.5,0.0,28.1,700.000000,15.050642,3.838692,301.0
6,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,7.0,810-P1004-S3,170.0,180.0,0.000000,-1697.369995,0.000000,219.099991,8.179,1.5,0.0,28.1,1697.369995,3.505798,3.665796,301.0
7,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,8.0,810-P1004-S3,180.0,190.0,0.000000,-289.588989,0.000000,219.099991,8.179,1.5,0.0,28.1,289.588989,3.665796,2.787860,301.0
8,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,9.0,810-P1004-S3,190.0,200.0,0.000000,-305.001007,0.000000,219.099991,8.179,1.5,0.0,28.1,305.001007,5.875957,6.953367,301.0
9,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,10.0,810-P1004-S3,200.0,210.0,-444.350006,0.000000,0.000000,219.099991,8.179,1.5,0.0,28.1,444.350006,2.709338,11.397946,301.0


In [212]:
segnum = list(set(df['SEG']))
segnum[4]

305.0

### Preprocessing

In [236]:
import math
import numpy as np

def value (i):
    if i < 0:
        i = -10
    elif i > 0 :
        i = 10
    
    return i

import numpy as np

def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1,v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

vectors_value =[]
print (np.shape(vectors_value))


(0,)


In [224]:
k=[]
for jj in range(len(segnum)):
    cropseg = df[df['SEG']==segnum[jj]] 
    newarr= np.array(cropseg)
    k.append(len(newarr))

maxnum = max(k)
print (maxnum)
# cropseg = df[df['SEG']==segnum[5]] 
# cropseg

67


In [237]:
for i in range(len(segnum)):
    
    
    cropseg = df[df['SEG']==segnum[i]] 
    newarr= np.array(cropseg)

    vec_value= np.zeros((1,maxnum))
#     print (newarr)
#     print (len(newarr))
    curr = 0
    if len(newarr)==1:
        x3 = value(newarr[0][6])
        y3 = value(newarr[0][7])
        z3 = value(newarr[0][8])    

        anglenum = angle_between((x3,y3,x3),(0,0,0))

        if (math.isnan(anglenum) is True):
            anglenum = -1

        vec_value[0,curr]= anglenum

    else:
        x3 = value(newarr[0][6])
        y3 = value(newarr[0][7])
        z3 = value(newarr[0][8])    

        anglenum = angle_between((x3,y3,x3),(0,0,0))

        if (math.isnan(anglenum) is True):
            anglenum = -1

        vec_value[0,curr]= anglenum


        for curr in range(len(newarr)-1):
            x3 = value(newarr[curr][6])
            y3 = value(newarr[curr][7])
            z3 = value(newarr[curr][8])

            x4 = value(newarr[curr+1][6])
            y4 = value(newarr[curr+1][7])
            z4 = value(newarr[curr+1][8])

            anglenum = angle_between((x3,y3,x3),(x4,y4,z4))

            if (math.isnan(anglenum) is True):
                anglenum = -1

            vec_value[0,curr+1]= anglenum

    vectors_value.append(vec_value)        
        

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [244]:
print (np.shape(vectors_value))

print (vectors_value[6629])

(6630, 1, 67)
[[-1.          0.          1.57079633  1.57079633  0.          0.
   0.          1.57079633  1.57079633  1.57079633 -1.          1.57079633
   1.04719755  0.95531662  0.78539816  0.78539816  2.35619449 -1.
  -1.          0.78539816  0.78539816  1.04719755  0.95531662  1.57079633
   1.57079633 -1.          1.57079633  1.57079633  0.          0.
   0.          1.57079633  1.57079633  1.23095942  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]]


In [248]:
vectors_value_new = np.zeros((len(vectors_value),maxnum))
                      
for kk in range(len(vectors_value)):
    vectors_value_new[kk,:]=vectors_value[kk][0]
                     

In [250]:
print (np.shape(vectors_value_new))

(6630, 67)


In [251]:
import umap
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# clusterable_embedding = umap.UMAP(
#     n_neighbors=30,
#     min_dist=0.0,
#     n_components=2,
#     random_state=42,
# ).fit_transform(digits.data)
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 50, 'linewidths':0}
plt.rcParams["figure.figsize"] = [9,7]

projection = TSNE().fit_transform(vectors_value_new)
plt.scatter(*projection.T, **plot_kwds)

clusterer = hdbscan.HDBSCAN(min_cluster_size=500).fit(vectors_value_new)
color_palette = sns.color_palette('Paired', 12)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]
plt.scatter(*projection.T, s=50, linewidth=0, c=cluster_member_colors, alpha=0.25)

ModuleNotFoundError: No module named 'umap'

### test detail

In [106]:
cropseg = df[df['SEG']==segnum[3]]
cropseg

newarr= np.array(cropseg)
# print (np.array(cropseg))
# print (len(cropseg))

# newarr = cropseg
print (newarr[])

,PROJECT,GUID,ELEMENTID,JOBNAME,FROM_NODE,TO_NODE,DELTA_X,DELTA_Y,DELTA_Z,DIAMETER,WALL_THICK,CORR_ALLOW,PRESSURE1,TEMP_MAX,LENGTH,MAXF,MAXT,SEG
90,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,91.0,810-P1004-S3,450.0,18110.0,0.000000,1003.650085,0.0,114.299995,6.0200,1.5,0.0,28.1,1003.650085,3.729838e+00,5.566140e-01,304.0
91,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,92.0,810-P1004-S3,18110.0,18120.0,0.000000,252.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,252.000000,5.566140e-01,5.765008e-01,304.0
92,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,93.0,810-P1004-S3,18120.0,18130.0,0.000000,0.000000,960.0,114.299995,6.0200,1.5,0.0,28.1,960.000000,6.766253e-01,1.090837e+00,304.0
93,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,94.0,810-P1004-S3,18130.0,18140.0,-3800.899658,0.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,3800.899658,1.174891e+00,1.366609e+00,304.0
94,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,95.0,810-P1004-S3,18140.0,18150.0,-542.849976,0.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,542.849976,1.366609e+00,1.525872e+00,304.0
95,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,96.0,810-P1004-S3,18150.0,18160.0,0.000000,2600.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,2600.000000,1.442493e+00,7.138289e-01,304.0
96,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,97.0,810-P1004-S3,18160.0,18170.0,0.000000,0.000000,740.0,114.299995,6.0200,1.5,0.0,28.1,740.000000,8.160173e-01,1.007856e+00,304.0
97,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,98.0,810-P1004-S3,18170.0,18180.0,0.000000,0.000000,2180.0,114.299995,6.0200,1.5,0.0,28.1,2180.000000,1.007856e+00,8.130978e-01,304.0
98,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,99.0,810-P1004-S3,18180.0,18185.0,0.000000,4743.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,4743.000000,7.851796e-01,2.528268e-01,304.0
99,SC6002,181ff4eb-bd1f-440a-aaf7-dfbd1575b793,100.0,810-P1004-S3,18185.0,18190.0,0.000000,3757.000000,0.0,114.299995,6.0200,1.5,0.0,28.1,3757.000000,2.528268e-01,4.522187e-02,304.0


In [130]:
def value (i):
    if i < 0:
        i = -10
    elif i > 0 :
        i = 10
    
    return i

import numpy as np

def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1,v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



newarr= np.array(cropseg)

newarr


array([['SC6002', '181ff4eb-bd1f-440a-aaf7-dfbd1575b793', 91.0,
        '810-P1004-S3', 450.0, 18110.0, 0.0, 1003.65008544922, 0.0,
        114.299995422363, 6.01999998092651, 1.5, 0.0, 28.1000003814697,
        1003.65008544922, 3.72983765602112, 0.55661404132843, 304.0],
       ['SC6002', '181ff4eb-bd1f-440a-aaf7-dfbd1575b793', 92.0,
        '810-P1004-S3', 18110.0, 18120.0, 0.0, 252.0, 0.0,
        114.299995422363, 6.01999998092651, 1.5, 0.0, 28.1000003814697,
        252.0, 0.55661404132843, 0.576500833034515, 304.0],
       ['SC6002', '181ff4eb-bd1f-440a-aaf7-dfbd1575b793', 93.0,
        '810-P1004-S3', 18120.0, 18130.0, 0.0, 0.0, 960.0,
        114.299995422363, 6.01999998092651, 1.5, 0.0, 28.1000003814697,
        960.0, 0.67662525177002, 1.09083676338196, 304.0],
       ['SC6002', '181ff4eb-bd1f-440a-aaf7-dfbd1575b793', 94.0,
        '810-P1004-S3', 18130.0, 18140.0, -3800.89965820313, 0.0, 0.0,
        114.299995422363, 6.01999998092651, 1.5, 0.0, 28.1000003814697,
        38

In [172]:
import math

maxnum = 67-1

vectors_value =[]
#    vectors_value.append(vec_value)
vec_value= np.zeros((1,maxnum))


for curr in range(len(newarr)-1):
#     x = newarr[curr][6]
#     y = newarr[curr][7]
#     z = newarr[curr][8]
    
#     x2 = newarr[curr+1][6]
#     y2 = newarr[curr+1][7]
#     z2 = newarr[curr+1][8]
    
#     print ("                       ",x,y,z,"                             ",x2,y2,z2)
#     print (angle_between((x2,y2,x2),(x,y,z)))
#     print (angle_between((x,y,z),(x2,y2,x2)))
   
    x3 = value(newarr[curr][6])
    y3 = value(newarr[curr][7])
    z3 = value(newarr[curr][8])
    
    x4 = value(newarr[curr+1][6])
    y4 = value(newarr[curr+1][7])
    z4 = value(newarr[curr+1][8])

    anglenum = angle_between((x3,y3,x3),(x4,y4,z4))
    
    if (math.isnan(anglenum) is True):
        anglenum = -1

    vec_value[0,curr]= anglenum
    
    
    
#     print (angle_between((x3,y3,x3),(x4,y4,z4)))
#     print (angle_between((x4,y4,z4),(x3,y3,x3)))

       
vectors_value.append(vec_value)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [173]:
print (np.shape(vectors_value))

(1, 1, 66)


1.5707963267948966

### max 값 67

In [68]:
maxnum = 67
vectors_value =[]
#    vectors_value.append(vec_value)
vec_value= np.zeros((1,maxnum))

print (vec_value.shape)
print (vec_value)

(1, 67)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [69]:
curr = 0 
x = cropseg['DELTA_X'][curr]
y = cropseg['DELTA_Y'][curr]
z = cropseg['DELTA_Z'][curr]

print (x,y,z)

KeyError: 0

In [25]:
x2 = cropseg['DELTA_X'][curr+1]
y2 = cropseg['DELTA_Y'][curr+1]
z2 = cropseg['DELTA_Z'][curr+1]

print (x2,y2,z2)

0.0 305.0 0.0


In [65]:
for curr in range(len(cropseg)-1):
    x = cropseg['DELTA_X'][curr]
    y = cropseg['DELTA_Y'][curr]
    z = cropseg['DELTA_Z'][curr]
    
    x2 = cropseg['DELTA_X'][curr+1]
    y2 = cropseg['DELTA_Y'][curr+1]
    z2 = cropseg['DELTA_Z'][curr+1]
    
    print ("                       ",x,y,z,"                             ",x2,y2,z2)
    print (angle_between((x,y,z),(x2,y2,x2)))
    
    

KeyError: 0